# SageMaker Edge Manager Raspberry Pi Example


## このサンプルの説明

- [こちらのブログ]()で使用しているサンプルになります。SageMaker Edge Managerのより詳しい説明はブログをご覧ください。
- このノートブックは[AWS公式のサンプ](https://github.com/aws/amazon-sagemaker-examples/tree/master/sagemaker_edge_manager)ルをもとに作成しています。前半のセットアップやモデルのパッケージングなどはサンプルとほぼ同じ内容です。そのため説明などを割愛しています
- 後半のエッジデバイスでの作業は本家サンプルではEC2で実行していましたが、こちらのサンプルではRaspberry Pi 4で実行します。

## そもそもSageMaker Edge Managerとは？

SageMaker Edge Managerとは、エッジデバイスに置いておく機械学習モデルの運用管理ツールです。

### これまでの課題

モデルをクラウド上でトレーニングして、どこかへデプロイするというパターンはこれまでもありましたが、特にエッジデバイスへのデプロイは場所の制約やスペックの制約があったり、数が多かったりと、管理が大変でした。

### SageMaker Edge Manager

SageMaker Edge Managerはデバイスをフリーととして管理、モデルをパッケージ化しデプロイすることができます。

## セットアップ

もろもろインポートします。

In [ ]:
import sagemaker
from sagemaker import get_execution_role
import boto3
import botocore
import json

role = get_execution_role()
sess = sagemaker.Session()
region = boto3.Session().region_name

In [ ]:
print(role)

上記で出力されたロールに以下のポリシーをアタッチしてください。AWS IoTの作業で必要となります。

- AWSIoTFullAccess 

In [ ]:
#フォルダ名はいい感じにしてください
bucket = sess.default_bucket() 
folder = 'DEMO-Sagemaker-Edge'
compilation_output_sub_folder = folder + '/compilation-output'
iot_folder = folder + '/iot'
s3_compilation_output_location = 's3://{}/{}'.format(bucket, compilation_output_sub_folder)

ファイルをアップロード

In [ ]:
darknet_img_path = sess.upload_data('darknet.bmp', bucket, iot_folder)
keras_img_path = sess.upload_data('keras.bmp', bucket, iot_folder)

## SageMaker Neoでモデルのコンパイル

SageMaker Neoは各ハードウェア向けにモデルを最適化する機能です。Edge Managerでパッケージ化してデプロイする前に、使用するデバイス向けにコンパイルしましょう。

In [ ]:
sagemaker_client = boto3.client('sagemaker', region_name=region)

### モデルのコンパイル
DarknetとKerasのモデルをコンパイルします。ちなみにモデルはDarknetが物体検出のモデルで、Kerasが画像分類のモデルです。

#### Darknetのモデルをダウンロード

In [ ]:
!wget -O yolov3-tiny.cfg https://github.com/pjreddie/darknet/blob/master/cfg/yolov3-tiny.cfg?raw=true
!wget https://pjreddie.com/media/files/yolov3-tiny.weights

In [ ]:
import tarfile

with tarfile.open('yolov3-tiny.tar.gz', mode='w:gz') as archive:
    archive.add('yolov3-tiny.cfg')
    archive.add('yolov3-tiny.weights')

In [ ]:
darknet_model_path = sess.upload_data('yolov3-tiny.tar.gz', bucket, folder)

In [ ]:
darknet_model_data_shape = '{"data":[1,3,416,416]}'
darknet_model_framework = 'darknet'
target_device = 'ml_c5'

In [ ]:
import time
darknet_compilation_job_name = 'Sagemaker-Edge-'+ str(time.time()).split('.')[0]
print('Compilation job for %s started' % darknet_compilation_job_name)

response = sagemaker_client.create_compilation_job(
        CompilationJobName=darknet_compilation_job_name,
        RoleArn=role,
        InputConfig={
            'S3Uri': darknet_model_path,
            'DataInputConfig': darknet_model_data_shape,
            'Framework': darknet_model_framework.upper()
        },
        OutputConfig={
            'S3OutputLocation': s3_compilation_output_location,
            'TargetDevice': target_device 
        },
        StoppingCondition={
            'MaxRuntimeInSeconds': 900
        }
    )

print(response)

# Poll every 30 sec
while True:
    response = sagemaker_client.describe_compilation_job(CompilationJobName=darknet_compilation_job_name)
    if response['CompilationJobStatus'] == 'COMPLETED':
        break
    elif response['CompilationJobStatus'] == 'FAILED':
        raise RuntimeError('Compilation failed')
    print('Compiling ...')
    time.sleep(30)
print('Done!')

#### Kerasのモデルをダウンロード

In [ ]:
import tensorflow as tf

model = tf.keras.applications.MobileNetV2()
model.save('mobilenet_v2.h5')

In [ ]:
import tarfile

with tarfile.open('mobilenet_v2.tar.gz', mode='w:gz') as archive:
    archive.add('mobilenet_v2.h5')

In [ ]:
keras_model_path = sess.upload_data('mobilenet_v2.tar.gz', bucket, folder)

In [ ]:
keras_model_data_shape = '{"input_1":[1,3,224,224]}'
keras_model_framework = 'keras'
target_device = 'ml_c5'

In [ ]:
import time
keras_compilation_job_name = 'Sagemaker-Edge-'+ str(time.time()).split('.')[0]
print('Compilation job for %s started' % keras_compilation_job_name)

response = sagemaker_client.create_compilation_job(
        CompilationJobName=keras_compilation_job_name,
        RoleArn=role,
        InputConfig={
            'S3Uri': keras_model_path,
            'DataInputConfig': keras_model_data_shape,
            'Framework': keras_model_framework.upper()
        },
        OutputConfig={
            'S3OutputLocation': s3_compilation_output_location,
            'TargetDevice': target_device 
        },
        StoppingCondition={
            'MaxRuntimeInSeconds': 900
        }
    )

print(response)

# Poll every 30 sec
while True:
    response = sagemaker_client.describe_compilation_job(CompilationJobName=keras_compilation_job_name)
    if response['CompilationJobStatus'] == 'COMPLETED':
        break
    elif response['CompilationJobStatus'] == 'FAILED':
        raise RuntimeError('Compilation failed')
    print('Compiling ...')
    time.sleep(30)
print('Done!')

## Sagemaker Edge Managerでパッケージ化



### Darknetのモデルのパッケージ化

In [ ]:
darknet_packaged_model_name = "darknet-model"
darknet_model_version = "1.0"
darknet_model_package = '{}-{}.tar.gz'.format(darknet_packaged_model_name, darknet_model_version)

In [ ]:
darknet_packaging_job_name=darknet_compilation_job_name+"-packaging"
response = sagemaker_client.create_edge_packaging_job(
    RoleArn=role,
    OutputConfig={
        'S3OutputLocation': s3_compilation_output_location,
    },
    ModelName=darknet_packaged_model_name,
    ModelVersion=darknet_model_version,
    EdgePackagingJobName=darknet_packaging_job_name,
    CompilationJobName=darknet_compilation_job_name,
)

print(response)

# Poll every 30 sec
while True:
    job_status = sagemaker_client.describe_edge_packaging_job(EdgePackagingJobName=darknet_packaging_job_name)
    if job_status['EdgePackagingJobStatus'] == 'COMPLETED':
        break
    elif job_status['EdgePackagingJobStatus'] == 'FAILED':
        raise RuntimeError('Edge Packaging failed')
    print('Packaging ...')
    time.sleep(30)
print('Done!')

In [ ]:
darknet_model_data = job_status["ModelArtifact"]

### Kerasのモデルのパッケージ化

In [ ]:
keras_packaged_model_name = "keras-model"
keras_model_version = "1.0"
keras_model_package = '{}-{}.tar.gz'.format(keras_packaged_model_name, keras_model_version)

In [ ]:
keras_packaging_job_name=keras_compilation_job_name+"-packaging"
response = sagemaker_client.create_edge_packaging_job(
    RoleArn=role,
    OutputConfig={
        'S3OutputLocation': s3_compilation_output_location,
    },
    ModelName=keras_packaged_model_name,
    ModelVersion=keras_model_version,
    EdgePackagingJobName=keras_packaging_job_name,
    CompilationJobName=keras_compilation_job_name,
)

print(response)

# Poll every 30 sec
while True:
    job_status = sagemaker_client.describe_edge_packaging_job(EdgePackagingJobName=keras_packaging_job_name)
    if job_status['EdgePackagingJobStatus'] == 'COMPLETED':
        break
    elif job_status['EdgePackagingJobStatus'] == 'FAILED':
        raise RuntimeError('Edge Packaging failed')
    print('Packaging ...')
    time.sleep(30)
print('Done!')

In [ ]:
keras_model_data = job_status["ModelArtifact"]

### AWS IoTで "モノ" を作成

SageMaker Edge Managerは、AWS IoT Coreを使ってデバイスの認証してEdge Managerのエンドポイントを呼び出します。 


In [ ]:
iot_client = boto3.client('iot', region_name=region)

In [ ]:
iot_thing_name = 'sm-edge-thing-raspi'
iot_thing_type = 'SagemakerEdgeSample'

In [ ]:
iot_client.create_thing_type(
    thingTypeName=iot_thing_type
)

In [ ]:
iot_client.create_thing(
    thingName=iot_thing_name,
    thingTypeName=iot_thing_type
)

### フリートの作成

デバイスフリートとはEdge Manager上でのデバイスのグループのようなもの

#### フリート用のIAMロール作成

フリートごとにIAMロールをアタッチする必要がある。

ポリシーは以下をアタッチ。

- AmazonSageMakerEdgeDeviceFleetPolicy
- AmazonSageMakerFullAccess
- AmazonS3FullAccess 
- AWSIoTFullAccess 

信頼関係は以下。
```
{
  "Version": "2012-10-17",
  "Statement": [
      {
        "Effect": "Allow",
        "Principal": {"Service": "credentials.iot.amazonaws.com"},
        "Action": "sts:AssumeRole"
      },
      {
        "Effect": "Allow",
        "Principal": {"Service": "sagemaker.amazonaws.com"},
        "Action": "sts:AssumeRole"
      }
  ]
}
```



In [ ]:
device_fleet_name ="sample-device-fleet"

sagemaker_client.create_device_fleet(
    DeviceFleetName=device_fleet_name,
    RoleArn='arn:aws:iam::119463424712:role/{上記で作成したロール名}',
    OutputConfig={
        'S3OutputLocation': s3_compilation_output_location
    }
)

#### デバイスをフリートに登録

デバイスをAWS IoT "モノ" と紐づけて、フリートへ登録します。

In [ ]:
device_name = "raspi-01" # device name should be 36 charactors

sagemaker_client.register_devices(
    DeviceFleetName=device_fleet_name,
    Devices=[
        {          
            "DeviceName": device_name,
            "IotThingName": iot_thing_name,
            "Description": "this is a sample raspi"
        }
    ]
)

### AWS IoTで証明書の作成

In [ ]:
iot_cert = iot_client.create_keys_and_certificate(
    setAsActive=True
)

Save the files and upload to S3 bucket, these files will be used to provide credentials on device to communicate with aws services.

In [ ]:
with open('./iot.pem.crt', 'w') as f:
    for line in iot_cert['certificatePem'].split('\n'):
        f.write(line)
        f.write('\n')

In [ ]:
with open('./iot_key.pem.key', 'w') as f:
    for line in iot_cert['keyPair']['PrivateKey'].split('\n'):
        f.write(line)
        f.write('\n')

In [ ]:
with open('./iot_key_pair.pem.key', 'w') as f:
    for line in iot_cert['keyPair']['PublicKey'].split('\n'):
        f.write(line)
        f.write('\n')

AWS IoT "モノ" がAWSリソースへアクセスするためにロールエイリアスが必要ですが、Edge Manager側でフリートを作成すると `SageMakerEdge-{フリート名}` でAWS IoTにロールエイリアスが作成される。

In [ ]:
role_alias_name = 'SageMakerEdge-' + device_fleet_name

role_alias = iot_client.describe_role_alias
(
    roleAlias=role_alias_name
)

このロールエイリアスを引き受けられるよう、証明書にポリシーを設定します。

In [ ]:
alias_policy = {
  "Version": "2012-10-17",
  "Statement": {
    "Effect": "Allow",
    "Action": "iot:AssumeRoleWithCertificate",
    "Resource": role_alias['roleAliasDescription']['roleAliasArn']
  }
}

In [ ]:
aliaspolicy = iot_client.create_policy(
    policyName='aliaspolicy',
    policyDocument=json.dumps(alias_policy),
)

In [ ]:
iot_client.attach_policy(
    policyName='aliaspolicy',
    target=iot_cert['certificateArn']
)

AWS IoTのエンドポイントを取得。

In [ ]:
iot_endpoint = iot_client.describe_endpoint(
    endpointType='iot:CredentialProvider'
)

In [ ]:
endpoint = "https://{}/role-aliases/{}/credentials".format(iot_endpoint['endpointAddress'], role_alias_name)

AWS公式のルートCAを入手。

In [ ]:
!wget https://www.amazontrust.com/repository/AmazonRootCA1.pem

## ラズパイで推論実行

ここからはラズベリーパイ4で実行する手順です。ラズパイにログインして実行してください。

このサンプル作成時には以下の環境で作成しています。

- Raspberry Pi 4B (8GB)
- OS: Raspberry Pi OS Lite

In [ ]:
ssm_client.send_command(
    InstanceIds=[instance_id],
    DocumentName="AWS-RunShellScript",
    OutputS3BucketName=bucket,
    OutputS3KeyPrefix=folder,
    Parameters={
        'commands':[
            "#!/bin/bash",
            "mkdir /demo",
            "aws s3 cp s3://sagemaker-edge-release-store-us-west-2-linux-x64/Releases/" + version + "/" + version + ".tgz demo.tgz",
            "tar -xf demo.tgz -C /demo",
            "cd /demo/bin",
            "chmod +x sagemaker_edge_agent_binary",
            "chmod +x sagemaker_edge_agent_client_example"
        ]
    }
)

Get model signing root certificates from release bucket.

In [ ]:
ssm_client.send_command(
    InstanceIds=[instance_id],
    DocumentName="AWS-RunShellScript",
    OutputS3BucketName=bucket,
    OutputS3KeyPrefix=folder,
    Parameters={
        'commands':[
            "#!/bin/bash",
            "cd /demo",
            "mkdir certificates",
            "aws s3 cp s3://sagemaker-edge-release-store-us-west-2-linux-x64/Certificates/" + region + "/" + region + ".pem certificates"
        ]
    }
)

Download IoT certificates and private key to EC2 instance. Download models and test images to EC2 instance.

In [ ]:
ssm_client.send_command(
    InstanceIds=[instance_id],
    DocumentName="AWS-RunShellScript",
    OutputS3BucketName=bucket,
    OutputS3KeyPrefix=folder,
    Parameters={
        'commands':[
            "#!/bin/bash",
            "cd /demo",
            "mkdir iot-credentials",
            "cd iot-credentials",
            "aws s3 cp " + root_ca_path + " .",  
            "aws s3 cp " + device_cert_path + " .",
            "aws s3 cp " + device_key_path + " .",
            "cd /demo",
            "aws s3 cp " + darknet_img_path + " .",
            "aws s3 cp " + darknet_model_data + " .",
            "mkdir darknet_model",
            "tar -xf " + darknet_model_package + " -C darknet_model",
            "aws s3 cp " + keras_img_path + " .",
            "aws s3 cp " + keras_model_data + " .",
            "mkdir keras_model",
            "tar -xf " + keras_model_package + " -C keras_model"
        ]
    }
)

#### Configure sagemaker edge manager agent

Generate sagemaker edge agent configuration file. 

In [ ]:
sagemaker_edge_config = {
    "sagemaker_edge_core_device_uuid": device_name,
    "sagemaker_edge_core_device_fleet_name": device_fleet_name,
    "sagemaker_edge_core_capture_data_buffer_size": 30,
    "sagemaker_edge_core_capture_data_batch_size": 10,
    "sagemaker_edge_core_capture_data_push_period_seconds": 4,
    "sagemaker_edge_core_folder_prefix": "demo_capture",
    "sagemaker_edge_core_region": region,
    "sagemaker_edge_core_root_certs_path": "/demo/certificates",
    "sagemaker_edge_provider_aws_ca_cert_file": "/demo/iot-credentials/AmazonRootCA1.pem",
    "sagemaker_edge_provider_aws_cert_file": "/demo/iot-credentials/iot.pem.crt",
    "sagemaker_edge_provider_aws_cert_pk_file": "/demo/iot-credentials/iot_key.pem.key",
    "sagemaker_edge_provider_aws_iot_cred_endpoint": endpoint,
    "sagemaker_edge_provider_provider": "Aws",
    "sagemaker_edge_provider_s3_bucket_name": bucket,
    "sagemaker_edge_core_capture_data_destination": "Cloud"
}

In [ ]:
edge_config_file = open("sagemaker_edge_config.json", "w")
json.dump(sagemaker_edge_config, edge_config_file, indent = 6)
edge_config_file.close()

Upload sagemaker edge agent configure to S3 bucket.

In [ ]:
config_path = sess.upload_data('sagemaker_edge_config.json', bucket, iot_folder)

Download sagemaker edge agent configure file to EC2 instance.

In [ ]:
ssm_client.send_command(
    InstanceIds=[instance_id],
    DocumentName="AWS-RunShellScript",
    OutputS3BucketName=bucket,
    OutputS3KeyPrefix=folder,
    Parameters={
        'commands':[
            "#!/bin/bash",
            "aws s3 cp " + config_path + ' /demo'
        ]
    }
)

#### Launch Sagemaker Edge Agent

In [ ]:
agent_out = ssm_client.send_command(
    InstanceIds=[instance_id],
    DocumentName="AWS-RunShellScript",
    OutputS3BucketName=bucket,
    OutputS3KeyPrefix=folder,
    Parameters={
        'commands':[
            "cd /demo",
            "rm -f /tmp/sagemaker_edge_agent_example.sock",
            "./bin/sagemaker_edge_agent_binary -a /tmp/sagemaker_edge_agent_example.sock -c sagemaker_edge_config.json" 
        ]
    }
)

In [ ]:
ssm_client.get_command_invocation(
    CommandId=agent_out['Command']['CommandId'],
    InstanceId=instance_id,
)

### Load Model

In this section, we show the model management capabilities offered by SageMaker Edge Manager. We will load the two compiled and packaged models with the SageMaker Edge Agent. This keeps both models ready to run inference with. As you will see, once the models are loaded you can run multiple inferences as many times as necessary until the models are unloaded. This relieves the client applications from the logic and operational burden of managing them separately. These models are now simply an API away from running inference with.

When loading the model with SageMaker Edge Agent, the argument to the API points the Agent to a directory containing the packaged model (without any extraneous files within the directory). 

#### Load darknet model

`darknet_model` is the path containing the packaged model in this notebook. `demo-darknet` is the name given to this model. This name will be used later to refer to this model for, making predictions, capturing data, unload.

In [ ]:
load_darknet_model_out = ssm_client.send_command(
    InstanceIds=[instance_id],
    DocumentName="AWS-RunShellScript",
    OutputS3BucketName=bucket,
    OutputS3KeyPrefix=folder,
    Parameters={
        'commands':[
            "cd /demo",
            "./bin/sagemaker_edge_agent_client_example LoadModel darknet_model demo-darknet"
        ]
    }
)

In [ ]:
ssm_client.get_command_invocation(
    CommandId=load_darknet_model_out['Command']['CommandId'],
    InstanceId=instance_id,
)

#### Load keras model

`keras_model` is the path containing the packaged model in this notebook. `demo-keras` is the name given to this model. This name will be used later to refer to this model for, making predictions, capturing data, unload.

In [ ]:
load_keras_model_out = ssm_client.send_command(
    InstanceIds=[instance_id],
    DocumentName="AWS-RunShellScript",
    OutputS3BucketName=bucket,
    OutputS3KeyPrefix=folder,
    Parameters={
        'commands':[
            "cd /demo",
            "./bin/sagemaker_edge_agent_client_example LoadModel keras_model demo-keras"
        ]
    }
)

In [ ]:
ssm_client.get_command_invocation(
    CommandId=load_keras_model_out['Command']['CommandId'],
    InstanceId=instance_id,
)

### List Models

This API simply lists all the models and their names that are loaded with SageMaker Edge Agent. Note that the names shown here are same as the ones provided during the LoadModel in the previous sections.

In [ ]:
list_model_out = ssm_client.send_command(
    InstanceIds=[instance_id],
    DocumentName="AWS-RunShellScript",
    OutputS3BucketName=bucket,
    OutputS3KeyPrefix=folder,
    Parameters={
        'commands':[
            "cd /demo",
            "./bin/sagemaker_edge_agent_client_example ListModels"
        ]
    }
)

In [ ]:
ssm_client.get_command_invocation(
    CommandId=list_model_out['Command']['CommandId'],
    InstanceId=instance_id,
)

### Run Predict

In this API, we pass the model name, input data file that will be directly fed into the neural network, input tensor name that was passed earlier during the compilation phase, along with it's size and shape.

#### Run prediction on darknet model

In [ ]:
darknet_predict_out = ssm_client.send_command(
    InstanceIds=[instance_id],
    DocumentName="AWS-RunShellScript",
    OutputS3BucketName=bucket,
    OutputS3KeyPrefix=folder,
    Parameters={
        'commands':[
            "cd /demo",
            "./bin/sagemaker_edge_agent_client_example Predict demo-darknet darknet.bmp data 416 416 3"
        ]
    }
)

In [ ]:
ssm_client.get_command_invocation(
    CommandId=darknet_predict_out['Command']['CommandId'],
    InstanceId=instance_id,
)

#### Run prediction on keras model

In [ ]:
keras_predict_out = ssm_client.send_command(
    InstanceIds=[instance_id],
    DocumentName="AWS-RunShellScript",
    OutputS3BucketName=bucket,
    OutputS3KeyPrefix=folder,
    Parameters={
        'commands':[
            "cd /demo",
            "./bin/sagemaker_edge_agent_client_example Predict demo-keras keras.bmp input_1 224 224 3"
        ]
    }
)

In [ ]:
ssm_client.get_command_invocation(
    CommandId=keras_predict_out['Command']['CommandId'],
    InstanceId=instance_id,
)

### Capture Data

Capture the inputs and outputs of an inference call to cloud or disk. The specific parameters were configured earlier in the config file. 

In [ ]:
darknet_capture_out = ssm_client.send_command(
    InstanceIds=[instance_id],
    DocumentName="AWS-RunShellScript",
    OutputS3BucketName=bucket,
    OutputS3KeyPrefix=folder,
    Parameters={
        'commands':[
            "cd /demo",
            "./bin/sagemaker_edge_agent_client_example PredictAndCapture demo-darknet darknet.bmp data 416 416 3"
        ]
    }
)

In [ ]:
ssm_client.get_command_invocation(
    CommandId=darknet_capture_out['Command']['CommandId'],
    InstanceId=instance_id,
)

In [ ]:
keras_capture_out = ssm_client.send_command(
    InstanceIds=[instance_id],
    DocumentName="AWS-RunShellScript",
    OutputS3BucketName=bucket,
    OutputS3KeyPrefix=folder,
    Parameters={
        'commands':[
            "cd /demo",
            "./bin/sagemaker_edge_agent_client_example PredictAndCapture demo-keras keras.bmp input_1 224 224 3"
        ]
    }
)

In [ ]:
ssm_client.get_command_invocation(
    CommandId=keras_capture_out['Command']['CommandId'],
    InstanceId=instance_id,
)

### Unload Model

After unloading a model, the same name can be reused for future `LoadModel` APIs calls.

In [ ]:
unload_model_out = ssm_client.send_command(
    InstanceIds=[instance_id],
    DocumentName="AWS-RunShellScript",
    OutputS3BucketName=bucket,
    OutputS3KeyPrefix=folder,
    Parameters={
        'commands':[
            "cd /demo",
            "./bin/sagemaker_edge_agent_client_example UnloadModel demo-darknet",
            "./bin/sagemaker_edge_agent_client_example UnloadModel demo-keras"
        ]
    }
)

In [ ]:
ssm_client.get_command_invocation(
    CommandId=unload_model_out['Command']['CommandId'],
    InstanceId=instance_id,
)

## Clean Up

Stop the Agent

In [ ]:
ssm_client.cancel_command(
    CommandId=agent_out['Command']['CommandId'],
    InstanceIds=[instance_id]
)

Stop the EC2 instance

In [ ]:
ec2_client.stop_instances(
    InstanceIds=[instance_id]
)

## Appendix

### (Optional)Install CloudWatch Agent 

In [ ]:
CW_log_config = {
      "agent": {
        "metrics_collection_interval": 10,
        "logfile": "/opt/aws/amazon-cloudwatch-agent/logs/amazon-cloudwatch-agent.log"
      },
      "logs": {
        "logs_collected": {
          "files": {
            "collect_list": [
              {
                "file_path": "/opt/aws/amazon-cloudwatch-agent/logs/amazon-cloudwatch-agent.log",
                "log_group_name": "amazon-cloudwatch-agent.log",
                "log_stream_name": "amazon-cloudwatch-agent.log",
                "timezone": "UTC"
              },
              {
                "file_path": "/opt/aws/amazon-cloudwatch-agent/logs/test.log",
                "log_group_name": "test.log",
                "log_stream_name": "test.log",
                "timezone": "Local"
              }
            ]
          }
        },
        "log_stream_name": "my_log_stream_name",
        "force_flush_interval" : 15
      }
}

In [ ]:
CW_file = open("cloudwatch.json", "w") 
json.dump(CW_log_config, CW_file, indent = 6) 
CW_file.close() 

In [ ]:
CW_config_path = sess.upload_data('cloudwatch.json', bucket, iot_folder)

In [ ]:
ssm_client.send_command(
    InstanceIds=[instance_id],
    DocumentName="AWS-RunShellScript",
    OutputS3BucketName=bucket,
    OutputS3KeyPrefix=folder,
    Parameters={
        'commands':[
            "#!/bin/bash",
            "aws s3 cp " + CW_config_path + " /opt/aws/amazon-cloudwatch-agent/etc/amazon-cloudwatch-agent.json"
        ]
    }
)

In [ ]:
ssm_client.send_command(
    InstanceIds=[instance_id],
    DocumentName="AWS-RunShellScript",
    OutputS3BucketName=bucket,
    OutputS3KeyPrefix=folder,
    Parameters={
        'commands':[
            "#!/bin/bash",
            "wget https://s3.amazonaws.com/amazoncloudwatch-agent/ubuntu/amd64/latest/amazon-cloudwatch-agent.deb",
            "sudo dpkg -i -E ./amazon-cloudwatch-agent.deb",
        ]
    }
)

Install Cloud Watch Agent to SSM agent.

In [ ]:
ssm_client.send_command(
    DocumentName="AWS-ConfigureAWSPackage",
    DocumentVersion='1',
    OutputS3BucketName=bucket,
    OutputS3KeyPrefix=folder,
    Targets=[
        {
            'Key': 'InstanceIds',
            'Values': [instance_id]
        },
    ],
    TimeoutSeconds=600,
    Parameters={
        'action': ['Install'],
        "name": ["AmazonCloudWatchAgent"]
    },
    MaxConcurrency='50',
    MaxErrors='0'
)

To debug with CloudWatch, add a paramater `CloudWatchOutputConfig` to `send_command`
```
CloudWatchOutputConfig={
    'CloudWatchOutputEnabled': True
}
```

Example:
```
ssm_client.send_command(
    InstanceIds=[instance_id],
    DocumentName="AWS-RunShellScript",
    OutputS3BucketName=bucket,
    OutputS3KeyPrefix=folder,
    CloudWatchOutputConfig={
        'CloudWatchOutputEnabled': True
    },
    Parameters={
        'commands':[
            "cd /demo",
            "./bin/neo_agent_binary -a /tmp/sagemaker_edge_agent_example.sock -c neo_config.json" 
        ]
    }
)
```

Running log can be found in cloud watch log group `/aws/ssm/AWS-RunShellScript`